In [5]:
%matplotlib qt                           
import numpy as np                    
import matplotlib as mp
import math
import matplotlib.pyplot as plt
import scipy
from scipy.stats import truncnorm

In [23]:
import numpy as np
import math
import matplotlib.pyplot as plt

class detector():
    def __init__(self, centre, side_len, rot, pix_len):
        self.centre = centre
        self.side_x = side_len[0]
        self.side_y = side_len[1]
        self.rot = math.radians(rot)
        self.pix_len_x = pix_len[0]
        self.pix_len_y = pix_len[1]
        self.pix_id = np.arange((side_len[0] / pix_len[0]) * (side_len[1] / pix_len[1]))
        self.pix_area = pix_len[0] * pix_len[1]
        self.pix_area_vect = np.array([math.sin(self.rot), 0, -math.cos(self.rot)]) * self.pix_area

        # Pixel center coordinates
        self.pix_x = np.linspace(
            self.centre[0] - (self.side_x / 2) * math.cos(self.rot),
            self.centre[0] + (self.side_x / 2) * math.cos(self.rot),
            int(self.side_x / self.pix_len_x)
        )
        self.pix_y = np.linspace(
            self.centre[1] - (self.side_y / 2),
            self.centre[1] + (self.side_y / 2),
            int(self.side_y / self.pix_len_y)
        )
        self.pix_z = np.linspace(
            self.centre[2] - (self.side_x / 2) * math.sin(self.rot),
            self.centre[2] + (self.side_x / 2) * math.sin(self.rot),
            int(self.side_x / self.pix_len_x)
        )

        # Meshgrid for 3D plotting
        self.pix_x_mesh, self.pix_y_mesh = np.meshgrid(self.pix_x, self.pix_y)
        self.pix_z_mesh, _ = np.meshgrid(self.pix_z, self.pix_y)

        self.pix_x_mesh_flat = self.pix_x_mesh.flatten()
        self.pix_y_mesh_flat = self.pix_y_mesh.flatten()
        self.pix_z_mesh_flat = self.pix_z_mesh.flatten()

    def plot_pix(self, color='b', marker='o'):
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(self.pix_x_mesh_flat, self.pix_y_mesh_flat, self.pix_z_mesh_flat, color=color, marker=marker)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        plt.show()

    def conv_1(self, t):
        return tuple(reversed(divmod(t, len(self.pix_x))))

    def conv_2(self, x, y):
        return y * len(self.pix_x) + x


In [28]:
# Common dimensions
side = [4, 4]           # cm
pix = [0.24, 0.24]      # cm

# Horizontal detector in XY plane
CZT_horiz = detector(
    centre=[-2, 0, -2],   # Center at z = -2
    side_len=side,
    rot=0,               # 0° rotation, XY plane
    pix_len=pix
)

# Vertical detector in YZ plane
CZT_vert = detector(
    centre=[0, 0, 0],   # Center at z = -2
    side_len=side,
    rot=90,              # 90° rotation for YZ orientation
    pix_len=pix
)


In [30]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Horizontal detector (XY plane) — Blue
ax.scatter(CZT_horiz.pix_x_mesh_flat, CZT_horiz.pix_y_mesh_flat, CZT_horiz.pix_z_mesh_flat, color='blue', marker='o', label='Horizontal CZT')

# Vertical detector (YZ plane) — Red
ax.scatter(CZT_vert.pix_x_mesh_flat, CZT_vert.pix_y_mesh_flat, CZT_vert.pix_z_mesh_flat, color='red', marker='s', label='Vertical CZT')

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend()
plt.title("Perpendicularly Arranged CZT Detectors")
plt.show()


In [ ]:
def sol_ang_cart(x, y, z, detector_obj):
    """
    Calculate the solid angles subtended by all pixels of `detector_obj` at point (x, y, z).

    Parameters:
        x, y, z         : Coordinates of the point of interest
        detector_obj    : Instance of the `detector` class

    Returns:
        solid_ang       : 2D array of solid angles for each pixel
        solid_ang_flat  : 1D flattened array of solid angles
    """
    vect_mesh = np.array([
        detector_obj.pix_x_mesh - x,
        detector_obj.pix_y_mesh - y,
        detector_obj.pix_z_mesh - z
    ])
    
    vect_mesh_flat = vect_mesh.reshape(3, 1, -1)  # Shape: (3, 1, num_pixels)
    proj_area = np.zeros(vect_mesh_flat.shape[2])
    solid_ang_flat = np.zeros(vect_mesh_flat.shape[2])
    
    for j in range(vect_mesh_flat.shape[2]):
        vect_f = vect_mesh_flat[:, 0, j]
        norm_vect = vect_f / np.linalg.norm(vect_f)
        proj_area[j] = np.dot(detector_obj.pix_norm, norm_vect)
        solid_ang_flat[j] = proj_area[j] / np.square(np.linalg.norm(vect_f))
    
    solid_ang = solid_ang_flat.reshape(detector_obj.pix_y_mesh.shape)
    return solid_ang, solid_ang_flat

def sol_ang_prob_cart(x, y, z, detector_obj):
    """
    If a photon is scattered from (x, y, z), calculate the probability distribution
    of it hitting any pixel of the given detector (e.g., CZT_vert or CZT_horiz).

    Returns:
        solid_angle_prob        : 2D array of hit probabilities per pixel
        solid_angle_prob_flat   : Flattened 1D array of probabilities
        maximum_prob            : Maximum probability value (most likely pixel)
    """
    solid_ang, solid_ang_flat = sol_ang_cart(x, y, z, detector_obj)
    
    solid_angle_prob = solid_ang / (4 * math.pi)
    solid_angle_prob_flat = solid_ang_flat / (4 * math.pi)
    maximum_prob = np.max(solid_angle_prob_flat)
    
    return solid_angle_prob, solid_angle_prob_flat, maximum_prob

def plot_sol_ang_prob_cart(x, y, z, detector_obj):
    """
    Plots the solid angle probability from (x, y, z) to each pixel of detector_obj.
    """
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    prob_matrix, _, _ = sol_ang_prob_cart(x, y, z, detector_obj)
    
    ax.plot_surface(detector_obj.pix_x_mesh, detector_obj.pix_y_mesh, prob_matrix,
                    rstride=1, cstride=1, cmap='viridis')
    
    ax.set_xlabel('Pixel X')
    ax.set_ylabel('Pixel Y')
    ax.set_zlabel('Solid Angle Probability')
    ax.set_title(f'Solid Angle Probabilities from ({x},{y},{z})')
    plt.show()

